# Taking Inferences from Standard Single-Dummy Solver

**Currently, most Single-Dummy Solvers (SDS) (Including my own) use something like the following process**
- Generate possible layouts of the opponents hands (I currently use ALL layouts for our SDS applied to a limited bridge game, but current SDS for regular bridge generate some number of random distributions)
- For each layout, run a double-dummy solver (DDS), and get the number of tricks made when playing each card in that layout
- Aggregate the results from all layouts, giving us the overall utility* of each card

*The simplest utility is simply the mean number of tricks (that's what my sds currently uses, but can be extended to the probability of making a contract or the total points score)

**This is not an optimal solution.**
Intuitively, this seems fine - we consider every possible layout, let both players play optimally in all of them, and aggregate the results. 
However, it soon becomes clear that this strategy for solving single dummy problems is not complete.

Let's look at an example using our SDS to illustrate where the issue arises

In [1]:
import numpy as np
import pandas as pd
import sys

from engine import Engine
import displayer
import simulator
from deal import Deal
from dds import DDS


In [2]:
game = Engine()

We first generate a (4 cards each) hand where it is south's turn

In [3]:
# We can either generate a hand automatically or manually input a hand.
# hand = {'N': ['C14', 'C13', 'C12', 'C11'], 'E': ['S14', 'S13', 'H12', 'H13'], 
    # 'S': ['D14', 'D11', 'H14', 'H11'], 'W': ['D12', 'D13', 'S12', 'S11']}
# deal1 = Deal(hand=hand, first_turn='S', trumps=None)
seed = 14
deal1 = Deal(seed=seed, cards_each=4, first_turn='S', trumps=None)
displayer.print_hands(deal1.current_hands)

		 S13 H12 D14    
		 S11            


    H14 D12 C14 		 S12     D13 C12
    H11         		             C11


		 S14 H13 D11 C13




We then get all the possible layouts of the hand, from the perspective of south (who can see their own hand and north's hand (the dummy)







































In [4]:
first_player = deal1.first_turn
first_layouts = simulator.find_layouts(deal1, first_player, on_lead=False)

print(f"There are {len(first_layouts)} possible layouts (8C4)")

There are 70 possible layouts (8C4)


We run our SDS on this position, which will run a DDS on each of these 70 layouts, and give us the mean number of tricks obtained for north south for each card they can play

In [5]:
first_df = game.sds(first_layouts, deal1)
mean_first_df = first_df.mean()
print(f"Average number of tricks expected for NS after {first_player} plays:")
print(mean_first_df)
first_card_played = mean_first_df.idxmax()
print("\nBest card:", first_card_played)

Average number of tricks expected for NS after S plays:
C13    1.171429
D11    2.000000
H13    1.028571
S14    3.000000
dtype: float64

Best card: S14


Lets play this best card, and move over to west and see what their options are.

In [6]:
deal1.play_card(deal1.first_turn, first_card_played)
displayer.print_hands(deal1.current_hands)


		 S13 H12 D14    
		 S11            


    H14 D12 C14 		 S12     D13 C12
    H11         		             C11


		     H13 D11 C13




Now, as west, we can use our SDS on the new position (accounting for the first card that was played)

In [7]:
second_player = deal1.play_order[deal1.current_turn_index]
second_layouts = simulator.find_layouts(deal1, second_player, on_lead=False)
print(f"There are {len(second_layouts)} possible layouts (7C3)\n")

second_df = game.sds(second_layouts, deal1)
mean_second_df = second_df.mean()
print(f"Average number of tricks expected for NS after {second_player} plays")
print(mean_second_df)
card_played = mean_second_df.idxmax()

There are 35 possible layouts (7C3)

Average number of tricks expected for NS after W plays
C14    3.400000
D12    2.800000
H11    2.914286
H14    3.342857
dtype: float64


Looks good. **However, we missed a crucial piece of information.** The fact that South 'chose' to play their first card, tells us information about their hand. So if we can work out what would entice South to play the card they played, then we can narrow down their distribution even more.

Luckily, we know exactly how South made their decision. Namely by using the SDS :)

Let's look at all of the possible information sets that South could have had when they made their decision, run our SDS on those information sets. 

Currently, we know that South could see all of dummy, and they could see the card they chose to play. We have already generated all the possible hands South could have, from the perspective of West, so we can use those again (the variable 'second_layouts')

In [8]:
likely_layouts = []
unlikely_layouts = []
for potential_original_layout in second_layouts:
    # Add the card South played back in to their hand
    potential_original_layout[first_player].insert(0, first_card_played)

    deal = Deal(hands=potential_original_layout, first_turn=first_player)
    # For this potential layout of the south cards, generate the layouts that EW could have
    layouts = simulator.find_layouts(deal, first_player, on_lead=False)

    potential_df = game.sds(layouts, deal) 
    mean_potential_df = potential_df.mean()
    # Get the card that SDS suggests to play
    current_card_played = mean_potential_df.idxmax()

    if current_card_played == first_card_played:
        likely_layouts.append(potential_original_layout)
    else:
        unlikely_layouts.append((potential_original_layout, current_card_played))


In [9]:
print(f"So there are only {len(likely_layouts)} layouts in which South would have played {first_card_played}")

So there are only 16 layouts in which South would have played S14


Now, as West, we know that South has one of those layouts. So, using those layouts, we can run our SDS and get a more accurate trick estimation for our decision

In [10]:
for i in likely_layouts:
    i[first_player].remove(first_card_played)

final_df = game.sds(likely_layouts, deal1)
mean_final_df = final_df.mean()
print(mean_final_df)

C14    3.4375
D12    2.5000
H11    2.7500
H14    3.3125
dtype: float64


So we can see that the expected value has changed once we make an inference from the fact that South played the card they did. Employing techniques like this should bring big improvements to SDS systems

### Thoughts/Issues

- In this reduced example above, we had to run the DDS 70*35 = 2450 times (70 times for each of the 35 possible hands south was holding). This very quickly becomes infeasible for problems with more cards than this. Note, however, that this SDS is 'complete', in that it calculates every possible layout, and does not reduce the size of the game tree (except for alpha-beta pruning). Reducing the game tree size is where the crux of the SDS is, and the use of heuristics created manually and learned from ILP/PILP will greatly help if integrated in this system.
- This strategy is only one-step. If North knows that West is employing this strategy, they can use that to aid their decisions. Moreover, if South knew that West would be employing this strategy, they could exploit it in their leading strategy. Thus, finding a true nash equilibrium is infeasible - but working towards one with heuristic simplifications could provide nice gains  
- Looking at the above point another way - while taking inference from the play can be hard, once it's taken, you can reduce the search tree size (we only searched 16 layouts in this example instead of the original 35). Although, you may want to still explore the whole tree and just give probabilistic weightings to our inferences.
- When playing against humans or a another AI, we won't be as certain about how they make their decisions, and so we are susceptible to being misled about their possible ranges. This is where assigning a probability to the inferences could be crucial
